<a href="https://colab.research.google.com/github/rerebebeccacca/20210602-machine-learing/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 2s 0us/step


In [2]:
dataset #tf.keras.utils.get_file 他是一個隱藏的資料夾

'/root/.keras/datasets/aclImdb.tar.gz'

In [3]:
import os
import glob
dn = os.path.split(dataset)[0]#把最後一段和資料夾名稱分開
fn = os.path.join(dn, "aclImdb", "train", "pos", "*")#加這個就不用再＊前面加斜線
#glob.glob(dn + "/*")
print(fn)
fns = glob.glob(fn)
with open(fns[0], "r", encoding="utf-8") as f:
    print(f.read())

/root/.keras/datasets/aclImdb/train/pos/*
I was told it was one of those "either you love it or you hate it" movies. Well, I loved it. Obvious hippie-era, dated and easy symbolism and all. So, I probably have no taste at all when it comes to Antonioni, but this and La Notte (made exactly a decade earlier) are my favourites among his movies so far. Made two years before I was born, Zabriskie Point was supposed to have been Michelangelo's great American epic. But apparently, it turned out to be a flop. I really can't see why. Before watching it I'd read that it was rather boring, so I braced myself for a very slow movie - though I love me a slow movie. For my taste, Zabriskie didn't have a tedious minute in it. While watching it, I made a mental note of how European it was on the director's part to make such frequent use of advertisement billboards in almost every urban scene, enormous billboards dwarfing any human form in sight. This recurrent visual element is obviously there to underl

In [4]:
import pandas as pd
def get_data(base):
    contents, target=[], []
    dn = os.path.split(dataset)[0]#把最後一段和資料夾名稱分開
    fn = os.path.join(dn, "aclImdb", base, "pos", "*.txt")#加這個就不用再＊前面加斜線
    fns = glob.glob(fn)
    for p in fns:
      with open(p, "r", encoding="utf-8") as f:
        contents.append(f.read())
        target.append(1)
    fn = os.path.join(dn, "aclImdb", base, "neg", "*.txt")
    fns = glob.glob(fn)
    for p in fns:
      with open(p, "r", encoding="utf-8") as f:
        contents.append(f.read())
        target.append(1)
    df = pd.DataFrame({
        "contents":contents,
        "ans":target
    })
    return df
train_df = get_data(base="train")
test_df = get_data(base="test")

In [5]:
test_df

,contents,ans
0,I was fortunate enough to see The Last Stop he...,1
1,I really appreciate what Jung-won had done bef...,1
2,"Very literate, intelligent drama about a group...",1
3,Even though I saw this film when I was very yo...,1
4,"""Nuovomondo"" was a great experience. Many film...",1
...,...,...
24995,I saw this on a screener DVD a couple months b...,1
24996,This movie has got to be the worse movie i hav...,1
24997,This is a piece of celluloid CRAP. You can tel...,1
24998,"I totally agree. This is ""Pitch Black undergro...",1


In [6]:
#step1. Tokenize
#我們絕對不會使用0，因為0是拿來做padding
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["contents"])

In [7]:
# Step2. Sequenize
#tok.word_index
#tok.index_word[3000]
#把文章（token)->轉成sequence
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])
pd.DataFrame(x_train_seq)#astype(int)轉成整數，但這邊不能用因為有缺失值

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,10,13,576,9,13,28,4,145,342.0,22.0,116.0,9.0,39.0,22.0,780.0,9.0,99.0,70.0,10.0,444.0,9.0,574.0,996.0,1957.0,2.0,772.0,2.0,29.0,35.0,10.0,239.0,25.0,54.0,1293.0,30.0,29.0,51.0,9.0,263.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,6,3,62,4,104,2512,2,3.0,1128.0,264.0,15.0,65.0,93.0,142.0,341.0,151.0,2.0,1559.0,2043.0,2682.0,182.0,295.0,577.0,2.0,1128.0,36.0,1.0,2.0,137.0,80.0,1.0,5.0,27.0,16.0,65.0,220.0,2718.0,1.0,1045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109,12,11,17,44,3,173,4,475.0,718.0,2425.0,10.0,101.0,9.0,141.0,46.0,50.0,1105.0,71.0,9.0,44.0,74.0,345.0,20.0,894.0,7.0,7.0,42.0,126.0,1117.0,221.0,9.0,755.0,705.0,16.0,380.0,2.0,585.0,18.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,6,32,318,19,15,665,642,665.0,202.0,448.0,10.0,101.0,12.0,1862.0,119.0,3.0,84.0,289.0,8.0,11.0,19.0,198.0,238.0,339.0,467.0,2.0,2078.0,5.0,27.0,3.0,84.0,56.0,44.0,3.0,52.0,1264.0,2.0,3.0,77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2444,70,90,659,20,10,25,207,421.0,11.0,1479.0,5.0,19.0,228.0,1005.0,41.0,3.0,50.0,1311.0,2.0,396.0,31.0,2234.0,1.0,1830.0,177.0,80.0,139.0,1077.0,2.0,488.0,5.0,341.0,7.0,7.0,10.0,241.0,249.0,88.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,11,44,32,218,640,917,922,111,18.0,42.0,1050.0,16.0,336.0,455.0,2.0,315.0,299.0,6.0,2270.0,464.0,1.0,580.0,4.0,3.0,864.0,62.0,2.0,455.0,16.0,38.0,673.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,2761,16,1,2538,6,3,725,47,705.0,12.0,112.0,183.0,5.0,166.0,98.0,2868.0,54.0,548.0,86.0,2731.0,174.0,2.0,1049.0,350.0,10.0,188.0,262.0,3.0,760.0,4.0,58.0,365.0,69.0,80.0,316.0,11.0,30.0,1.0,99.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,832,111,2,104,2041,226,90,102,165.0,37.0,602.0,2961.0,5.0,132.0,11.0,90.0,9.0,875.0,5.0,231.0,15.0,98.0,4.0,1.0,102.0,258.0,1.0,26.0,605.0,2.0,914.0,50.0,37.0,3.0,1069.0,8.0,2733.0,10.0,479.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,1,83,317,4,11,307,13,1,115.0,204.0,107.0,2.0,10.0,101.0,204.0,107.0,172.0,307.0,4.0,1012.0,123.0,90.0,1.0,940.0,4.0,1546.0,2.0,106.0,68.0,92.0,9.0,13.0,14.0,148.0,291.0,298.0,446.0,11.0,6.0,617.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
SEQ_LEN = 512

In [9]:
# Step3. Padding #截長補短
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=SEQ_LEN)
x_test_pad = pad_sequences(x_test_seq, maxlen=SEQ_LEN)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,123,811,14,227,14,91,268,18,10,188,64,86,256,67,91,932,2,386,278,4,916,763,112,44,32,605,35,49,2,35,9,183,5,27,32,12,657,244,71,2338
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,64,620,31,1,1867,2,56,183,37,3,1,2120,133,6,2,160,577,44,54,577,16,12,191,2120,2,24,2161,16,1,269,1347,11,6,46,84,250,15,1,223,220
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,938,22,205,5,1,127,10,37,131,2567,4,99,2,10,431,101,11,28,889,1,1270,7,7,5,69,9,13,287,146,50,71,277,35,634,199,9,32,706,20,894
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,120,175,1,115,56,67,78,10,63,506,12,19,2,293,16,1,169,339,3,244,354,751,2,467,3,73,50,218,751,71,339,48,3,2233,9,1478,615,243,20,285
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,123,1244,3,2211,90,8,1,227,2916,11,141,199,22,139,5,101,41,7,7,1335,2,542,1174,356,5,64,50,37,11,5,29,4,145,571,8,1,228,4,11,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11,44,32,218,640,917,922,111,18,42,1050,16,336,455,2,315,299,6,2270,464,1,580,4,3,864,62,2,455,16,38,673
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,2312,39,1,52,854,236,36,2688,1,694,174,583,2330,2,1839,2546,78,114,5,335,7,7,47,63,215,3,173,5,132,40,199,9,3,715,7,7,2712,2433,434
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,430,339,378,15,49,225,1,1139,13,63,2697,10,216,9,20,2666,10,158,987,49,275,5,64,9,30,3,747,143,128,3,114,2416,30,86,108,84,853,11,17,44
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,766,47,13,35,114,39,57,133,940,12,9,59,27,875,15,256,21,1077,16,1,62,57,5,791,1,191,1386,13,12,1,451,35,73,437,2,92,1,127,9


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
    # 每個詞彙都有128個係數, 3001詞 * 128種 = 384128
    # !!!tokenize有多少個可調(上面要記得調): 3000 -> 4000
    # !!!詞向量維度可調(簡單小一點, 困難大一點): 128 -> 256/64
    # !!!input_length(上面要記得調): 512 -> 256
    Embedding(3001, 128, mask_zero=True, input_length=SEQ_LEN),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=["accuracy"])

In [ ]:
import numpy as np
y_train = np.array(train_df["ans"])
y_test = np.array(test_df["ans"])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# validation_split: 切出一些驗證資料來做驗證(0.1: 54000筆資料訓練)
# epochs: 看幾輪(10輪: 54000 * 10)
# batch_size: 看了多少筆再做一次梯度調整(看圖片大小, 圖片小(20*20):200, 圖片大(200*200):20)
# 200batch, 10epoch: 54000 * 10 / 200 -> 2700(梯度調整)
# verbose: 0(無聲) 1(最詳細) 2(沒有進度條)
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("model.h5", save_best_only=True)
]
# h5: https://www.hdfgroup.org/downloads/hdfview/
model.fit(x_train_pad,
          y_train,
          validation_split=0.1,
          epochs=100,
          batch_size=200,
          verbose=2,
          callbacks=callbacks)

Epoch 1/100
113/113 - 7s - loss: 0.2300 - accuracy: 0.9969 - val_loss: 0.0272 - val_accuracy: 1.0000
Epoch 2/100
113/113 - 3s - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.0063 - val_accuracy: 1.0000
Epoch 3/100
113/113 - 3s - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 4/100
113/113 - 3s - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 5/100
113/113 - 3s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 6/100
113/113 - 3s - loss: 9.0860e-04 - accuracy: 1.0000 - val_loss: 7.5743e-04 - val_accuracy: 1.0000
Epoch 7/100
113/113 - 3s - loss: 6.5775e-04 - accuracy: 1.0000 - val_loss: 5.6295e-04 - val_accuracy: 1.0000
Epoch 8/100
113/113 - 3s - loss: 4.9826e-04 - accuracy: 1.0000 - val_loss: 4.3460e-04 - val_accuracy: 1.0000
Epoch 9/100
113/113 - 3s - loss: 3.9025e-04 - accuracy: 1.0000 - val_loss: 3.4531e-04 - val_accuracy: 1.0000
Epoch 10/100
113/113 - 3s - loss: 3.1359e-04 - accuracy: 1.

In [14]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 2.8500e-07 - accuracy: 1.0000


[2.8500079451987403e-07, 1.0]

In [15]:
layers = [
    # 每個詞彙都有128個係數, 3001 * 128 = 384128
    # !!!tokenize有多少個可調(上面要記得調): 3000 -> 4000
    # !!!詞向量維度可調(簡單小一點, 困難大一點): 128 -> 256/64
    # !!!input_length(上面要記得調): 512 -> 256
    Embedding(3001, 128, mask_zero=True)
]
w2v = Sequential(layers)
w2v.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [16]:
w = model.layers[0].get_weights()
w2v.layers[0].set_weights(w)

In [17]:
from scipy.spatial.distance import cosine
from numpy.linalg import norm

tok1 = tok.word_index["scary"]
word1 = [[tok1]]
v1 = w2v.predict(word1)[0][0]
# print(v1)
tok2 = tok.word_index["horror"]
word2 = [[tok2]]
v2 = w2v.predict(word2)[0][0]
# 相似度 scipy: 1 - cos -> 1 - (1 - cos)
# cos: -1反向相似 0不相干 1正面相似 0.5以上 蠻像的兩個詞
print(1 - cosine(v1, v2))
print(np.dot(v1, v2) / (norm(v1) * norm(v2)))

0.9623875617980957
0.96238756
